In [1]:
import requests
import numpy as np
import pandas as pd

In [2]:
import base64
def get_token(client_id, client_secret):
    '''
    Function that receives client_id and client_secret from the Spotify app 
    and returns a valid token for using with the API.
    '''
    encoded = base64.b64encode(bytes(client_id+':'+client_secret, 'utf-8'))
    params = {'grant_type':'client_credentials'}
    header={'Authorization': 'Basic ' + str(encoded, 'utf-8')}
    r = requests.post('https://accounts.spotify.com/api/token', headers=header, data=params)
    if r.status_code != 200:
        print('Error en la request.', r.json())
        return None
    print('Token valid for {} seconds.'.format(r.json()['expires_in']))
    return r.json()['access_token']

def search_artist_id(string, token):
    '''
    Receives a search string and a valid token and returns the most popular artist matching that string, 
    including artist ID. If the search has 0 results, the return value is (string, None) 
    '''
    ep_busqueda = 'https://api.spotify.com/v1/search'
    header = {'Authorization': f'Bearer {token}'}
    search_params = {'q': string, 'type': 'artist'}#, 'market': 'AR'}
    r = requests.get(ep_busqueda, headers=header, params=search_params)
    if r.status_code == 200:
        if len(r.json()['artists']['items']) > 0:
            return tuple(pd.DataFrame(r.json()['artists']['items']).sort_values(by='popularity', ascending=False).loc[0,['name', 'id']].values)
        else:
            print('Artist not found:', string)
            return (string, None)

    else:
        print("Error during the request")
        print(r.status_code)
        print(r.text)
        
        
def get_artist_info(artist_id, token):
    '''
    Receives artist_id and a valid token and returns amount of followers, genres and popularity of the artist
    '''
    header = {'Authorization': f'Bearer {token}'}
    r = requests.get(f'https://api.spotify.com/v1/artists/{artist_id}', headers=header)
    if r.status_code == 200:
        return {'followers': r.json()['followers']['total'], 
                'genres': r.json()['genres'], 
                'popularity': r.json()['popularity']}
    else:
        print("Error during the request")
        print(r.status_code)
        print(r.text)
        
def get_related_artists(artist_id, token):
    '''
    Receives artist_id and a valid token and returns a list of related artists and a list of related genres,
    with possible repeated values.
    '''
    header = {'Authorization': f'Bearer {token}'}
    r = requests.get(f'https://api.spotify.com/v1/artists/{artist_id}/related-artists', headers=header)
    if r.status_code == 200:
        genres = []
        for genre_list in [artist['genres'] for artist in r.json()['artists']]:
            genres = genres+ genre_list
        return {'related_artists': [artist['name'] for artist in r.json()['artists']], 
                'related_genres':genres}
    else:
        print("Error during the request")
        print(r.status_code)
        print(r.text)

Read the credentials from 'credentials.txt'

In [3]:
client_id = None
client_secret = None
with open('credentials.txt', 'r') as file:
    for line in file:
        if 'client_id' in line:
            start = line.find('\'')+1
            end = line[start:].find('\'')
            client_id = line[start:start+end]
            print('Found Client ID')#, client_id)
        elif 'client_secret' in line:
            start = line.find('\'')+1
            end = line[start:].find('\'')
            client_secret = line[start:start+end]
            print('Found Client secret')#, client_secret)
        if client_id != None and client_secret != None:
            break
    file.close()
    if client_id == None:
        print('Client ID not found in credentials.txt')
    if client_secret == None:
        print('Client secret not found in credentials.txt')

Found Client ID
Found Client secret


Get a valid token based on the credentials read

In [4]:
token = get_token(client_id, client_secret)

Token valid for 3600 seconds.


Lollapalooza lineup from https://www.lollapaloozaar.com/news/lollapalooza-argentina-confirma-su-lineup-por-dia/

In [5]:
lineup_day_1 = 'Travis Scott – Martin Garrix – Los Fabulosos Cadillacs – Brockhampton – DUKI – Madeon – Rita Ora – A Day to Remember – King Princess – LP – WOS – Denzel Curry – Yungblud – Nathy Peluso – J mena – AJR – Louta – Two Feet – Bizarrap – Fuego – La Delio Valdez – Dani – Kaydy Cain – Maye – Boombox Cartel – Cimafunk – Ms Nina – Feli Colina – Axel Fiks – Lucia Tacchetti – DJ Sky – Alejo y Valentin'
lineup_day_2 = 'The Strokes – Gwen Stefani – Armin Van Buuren – Vampire Weekend – Ratones Paranoicos – ILLENIUM – Kacey Musgraves – Litto Nebbia – Charli XCX – Jaden Smith – R3HAB – Hayley Kiyoko – El Mató a un Policía Motorizado – Rels B – Emmanuel Horvilleur – Kali Uchis – Paloma Mami – Fabiana Cantilo – Trueno – Amaia – Wallows – Yung Beef – Zoe Gotusso – Girl Ultra – Las Ligas Menores – Ainda – Ghetto Kids – D3FAI – Paco Leiva – LIMON – Louly'
lineup_day_3 = 'Guns N’ Roses – Lana del Rey – Cage the Elephant – James Blake – Alan Walker – Rezz – The Lumineers – Rex Orange County – MIKA – Perry Farrell’s Kind Heaven Orchestra – Airbag – Lauv – YSY A – Chris Lake – Pabllo Vittar – San Holo – Emilia – Idles – Masego – The Hu – Elsa y Elmar – Natalie Perez – Goldfish – Florian – Miranda Johansen – DABOW – El Buen Salvaje – Metro Live – Reydel'

Convert the string into a list of artists

In [6]:
str_to_list = lambda x: [band.strip() for band in x.split('–')]

In [7]:
lineup_day_1 = str_to_list(lineup_day_1)
lineup_day_2 = str_to_list(lineup_day_2)
lineup_day_3 = str_to_list(lineup_day_3)

Find artists ids on Spotify

In [8]:
artists_ids = [search_artist_id(artist, token) for artist in lineup_day_1+lineup_day_2+lineup_day_3]

Artist not found: Perry Farrell’s Kind Heaven Orchestra
Artist not found: Metro Live


In [9]:
# Make a dict from list of tuples
artists_ids = {artist_name: artist_id for (artist_name, artist_id) in artists_ids}

Manually search for a missing artist's id

In [10]:
search_artist_id('perry farrell', token)

('Perry Farrell', '2DS5RPK5A2GQTucYlgsQdE')

In [11]:
artists_ids['Perry Farrell’s Kind Heaven Orchestra'] = search_artist_id('perry farrell', token)[1]

Remove "Metro Live" since it's not an artist.

In [12]:
artists_ids.pop('Metro Live')

## Building the artists DataFrame

In [13]:
df_artists = pd.DataFrame.from_dict(artists_ids, orient='index', columns=['artist_id'])
df_artists

,artist_id
Travis Scott,0Y5tJX1MQlPlqiwlOH1tJY
Martin Garrix,60d24wfXkVzDSfLS6hyCjZ
Los Fabulosos Cadillacs,2FS22haX3FYbyOsUAkuYqZ
BROCKHAMPTON,1Bl6wpkWCQ4KVgnASpvzzA
Duki,1bAftSH8umNcGZ0uyV7LMg
...,...
Florian Picasso,4GWqzTTt2uA9Ms6HfUhWUn
Miranda Johansen,6OqKESKxY3ltCGxUnWBrAX
Dabow,6GQx1heM2wyI3iHpAC5P8X
El Buen Salvaje,4N8uvTc9wjbwxaKBEwae7C


Get info for each artist and add it to the df

In [14]:
df_artists[['followers', 'genres', 'popularity']] = df_artists.apply(lambda x: get_artist_info(x['artist_id'], token=token), axis=1, result_type='expand')
df_artists

,artist_id,followers,genres,popularity
Travis Scott,0Y5tJX1MQlPlqiwlOH1tJY,7663003,[rap],98
Martin Garrix,60d24wfXkVzDSfLS6hyCjZ,12531782,"[big room, edm, pop, progressive house, tropic...",88
Los Fabulosos Cadillacs,2FS22haX3FYbyOsUAkuYqZ,1668356,"[argentine rock, latin alternative, latin rock...",73
BROCKHAMPTON,1Bl6wpkWCQ4KVgnASpvzzA,1174109,"[boy band, hip hop, rap]",85
Duki,1bAftSH8umNcGZ0uyV7LMg,2700103,"[argentine hip hop, trap argentino]",84
...,...,...,...,...
Florian Picasso,4GWqzTTt2uA9Ms6HfUhWUn,48516,"[big room, deep big room, edm, electro house, ...",55
Miranda Johansen,6OqKESKxY3ltCGxUnWBrAX,1998,[],42
Dabow,6GQx1heM2wyI3iHpAC5P8X,5651,[electronic trap],37
El Buen Salvaje,4N8uvTc9wjbwxaKBEwae7C,789,[],10


In [15]:
df_artists[['related_artists', 'related_genres']] = df_artists.apply(lambda x: get_related_artists(x['artist_id'], token=token), axis=1, result_type='expand')
df_artists

,artist_id,followers,genres,popularity,related_artists,related_genres
Travis Scott,0Y5tJX1MQlPlqiwlOH1tJY,7663003,[rap],98,"[A$AP Rocky, Chief Keef, KIDS SEE GHOSTS, Push...","[east coast hip hop, hip hop, pop rap, rap, tr..."
Martin Garrix,60d24wfXkVzDSfLS6hyCjZ,12531782,"[big room, edm, pop, progressive house, tropic...",88,"[AREA21, Brooks, Deorro, Lucas & Steve, KAAZE,...","[big room, edm, big room, deep big room, edm, ..."
Los Fabulosos Cadillacs,2FS22haX3FYbyOsUAkuYqZ,1668356,"[argentine rock, latin alternative, latin rock...",73,"[Los Pericos, Vicentico, Aterciopelados, Los A...","[argentine reggae, argentine rock, latin alter..."
BROCKHAMPTON,1Bl6wpkWCQ4KVgnASpvzzA,1174109,"[boy band, hip hop, rap]",85,"[Kevin Abstract, Matt Champion, JPEGMAFIA, Ame...","[hip hop, lgbtq+ hip hop, rap, bedroom pop, al..."
Duki,1bAftSH8umNcGZ0uyV7LMg,2700103,"[argentine hip hop, trap argentino]",84,"[Uriel Natero, Bhavi, Khea, Lit Killah, Seven ...","[argentine hip hop, trap argentino, argentine ..."
...,...,...,...,...,...,...
Florian Picasso,4GWqzTTt2uA9Ms6HfUhWUn,48516,"[big room, deep big room, edm, electro house, ...",55,"[Paris Blohm, Stadiumx, Brooks, Dannic, Jay Ha...","[big room, deep big room, edm, electro house, ..."
Miranda Johansen,6OqKESKxY3ltCGxUnWBrAX,1998,[],42,"[Feli Colina, Los Besos, Chita, Weste, FLORIAN...","[argentine indie, argentine indie, argentine i..."
Dabow,6GQx1heM2wyI3iHpAC5P8X,5651,[electronic trap],37,"[Juelz, Ian Munro, sfam, Lucchii, Untitld, Dil...","[canadian electronic, nz electronic, experimen..."
El Buen Salvaje,4N8uvTc9wjbwxaKBEwae7C,789,[],10,"[BIGGER, MALMAN, Cazacuervos, REYDEL, LIMON, P...","[argentine alternative rock, argentine alterna..."


### Data wrangling 

In [16]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb.fit(df_artists['genres'])

MultiLabelBinarizer(classes=None, sparse_output=False)

In [17]:
df_genres = pd.DataFrame(data=mlb.transform(df_artists['genres']), 
                         columns=['genre: {}'.format(genre) for genre in mlb.classes_],
                         index=df_artists.index)
df_genres

,genre: alternative r&b,genre: alternative rock,genre: argentine alternative rock,genre: argentine hip hop,genre: argentine indie,genre: argentine indie rock,genre: argentine rock,genre: argentine telepop,genre: art pop,genre: baile pop,...,genre: trap argentino,genre: trap chileno,genre: trap espanol,genre: trap latino,genre: tropical,genre: tropical house,genre: uk alternative pop,genre: uk pop,genre: underground hip hop,genre: vapor twitch
Travis Scott,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Martin Garrix,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
Los Fabulosos Cadillacs,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BROCKHAMPTON,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Duki,0,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Florian Picasso,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Miranda Johansen,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Dabow,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
El Buen Salvaje,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
df_artists = df_artists.join(df_genres)
df_artists

,artist_id,followers,genres,popularity,related_artists,related_genres,genre: alternative r&b,genre: alternative rock,genre: argentine alternative rock,genre: argentine hip hop,...,genre: trap argentino,genre: trap chileno,genre: trap espanol,genre: trap latino,genre: tropical,genre: tropical house,genre: uk alternative pop,genre: uk pop,genre: underground hip hop,genre: vapor twitch
Travis Scott,0Y5tJX1MQlPlqiwlOH1tJY,7663003,[rap],98,"[A$AP Rocky, Chief Keef, KIDS SEE GHOSTS, Push...","[east coast hip hop, hip hop, pop rap, rap, tr...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Martin Garrix,60d24wfXkVzDSfLS6hyCjZ,12531782,"[big room, edm, pop, progressive house, tropic...",88,"[AREA21, Brooks, Deorro, Lucas & Steve, KAAZE,...","[big room, edm, big room, deep big room, edm, ...",0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
Los Fabulosos Cadillacs,2FS22haX3FYbyOsUAkuYqZ,1668356,"[argentine rock, latin alternative, latin rock...",73,"[Los Pericos, Vicentico, Aterciopelados, Los A...","[argentine reggae, argentine rock, latin alter...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BROCKHAMPTON,1Bl6wpkWCQ4KVgnASpvzzA,1174109,"[boy band, hip hop, rap]",85,"[Kevin Abstract, Matt Champion, JPEGMAFIA, Ame...","[hip hop, lgbtq+ hip hop, rap, bedroom pop, al...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Duki,1bAftSH8umNcGZ0uyV7LMg,2700103,"[argentine hip hop, trap argentino]",84,"[Uriel Natero, Bhavi, Khea, Lit Killah, Seven ...","[argentine hip hop, trap argentino, argentine ...",0,0,0,1,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Florian Picasso,4GWqzTTt2uA9Ms6HfUhWUn,48516,"[big room, deep big room, edm, electro house, ...",55,"[Paris Blohm, Stadiumx, Brooks, Dannic, Jay Ha...","[big room, deep big room, edm, electro house, ...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Miranda Johansen,6OqKESKxY3ltCGxUnWBrAX,1998,[],42,"[Feli Colina, Los Besos, Chita, Weste, FLORIAN...","[argentine indie, argentine indie, argentine i...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Dabow,6GQx1heM2wyI3iHpAC5P8X,5651,[electronic trap],37,"[Juelz, Ian Munro, sfam, Lucchii, Untitld, Dil...","[canadian electronic, nz electronic, experimen...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
El Buen Salvaje,4N8uvTc9wjbwxaKBEwae7C,789,[],10,"[BIGGER, MALMAN, Cazacuervos, REYDEL, LIMON, P...","[argentine alternative rock, argentine alterna...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
mlb.fit(df_artists['related_artists'])
df_related_artists = pd.DataFrame(data=mlb.transform(df_artists['related_artists']), 
                         columns=['related_artist: {}'.format(related_artist) for related_artist in mlb.classes_],
                         index=df_artists.index)
df_related_artists

,related_artist: #TocoParaVos,related_artist: 0-600,related_artist: 070 Shake,related_artist: 100 gecs,related_artist: 107 Faunos,related_artist: 1788-L,related_artist: 1915,related_artist: 21 Savage,related_artist: 3LAU,related_artist: 8 Graves,...,related_artist: mimi bay,related_artist: sfam,related_artist: slowthai,related_artist: sumthin sumthin,related_artist: t.A.T.u.,related_artist: thook,related_artist: tobi lou,related_artist: zack villere,related_artist: Árbol,related_artist: Ñejo
Travis Scott,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Martin Garrix,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Los Fabulosos Cadillacs,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BROCKHAMPTON,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
Duki,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Florian Picasso,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Miranda Johansen,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Dabow,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,1,0,0,0,0
El Buen Salvaje,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
df_artists = df_artists.join(df_related_artists)
df_artists

,artist_id,followers,genres,popularity,related_artists,related_genres,genre: alternative r&b,genre: alternative rock,genre: argentine alternative rock,genre: argentine hip hop,...,related_artist: mimi bay,related_artist: sfam,related_artist: slowthai,related_artist: sumthin sumthin,related_artist: t.A.T.u.,related_artist: thook,related_artist: tobi lou,related_artist: zack villere,related_artist: Árbol,related_artist: Ñejo
Travis Scott,0Y5tJX1MQlPlqiwlOH1tJY,7663003,[rap],98,"[A$AP Rocky, Chief Keef, KIDS SEE GHOSTS, Push...","[east coast hip hop, hip hop, pop rap, rap, tr...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Martin Garrix,60d24wfXkVzDSfLS6hyCjZ,12531782,"[big room, edm, pop, progressive house, tropic...",88,"[AREA21, Brooks, Deorro, Lucas & Steve, KAAZE,...","[big room, edm, big room, deep big room, edm, ...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Los Fabulosos Cadillacs,2FS22haX3FYbyOsUAkuYqZ,1668356,"[argentine rock, latin alternative, latin rock...",73,"[Los Pericos, Vicentico, Aterciopelados, Los A...","[argentine reggae, argentine rock, latin alter...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BROCKHAMPTON,1Bl6wpkWCQ4KVgnASpvzzA,1174109,"[boy band, hip hop, rap]",85,"[Kevin Abstract, Matt Champion, JPEGMAFIA, Ame...","[hip hop, lgbtq+ hip hop, rap, bedroom pop, al...",0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
Duki,1bAftSH8umNcGZ0uyV7LMg,2700103,"[argentine hip hop, trap argentino]",84,"[Uriel Natero, Bhavi, Khea, Lit Killah, Seven ...","[argentine hip hop, trap argentino, argentine ...",0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Florian Picasso,4GWqzTTt2uA9Ms6HfUhWUn,48516,"[big room, deep big room, edm, electro house, ...",55,"[Paris Blohm, Stadiumx, Brooks, Dannic, Jay Ha...","[big room, deep big room, edm, electro house, ...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Miranda Johansen,6OqKESKxY3ltCGxUnWBrAX,1998,[],42,"[Feli Colina, Los Besos, Chita, Weste, FLORIAN...","[argentine indie, argentine indie, argentine i...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Dabow,6GQx1heM2wyI3iHpAC5P8X,5651,[electronic trap],37,"[Juelz, Ian Munro, sfam, Lucchii, Untitld, Dil...","[canadian electronic, nz electronic, experimen...",0,0,0,0,...,0,1,0,1,0,1,0,0,0,0
El Buen Salvaje,4N8uvTc9wjbwxaKBEwae7C,789,[],10,"[BIGGER, MALMAN, Cazacuervos, REYDEL, LIMON, P...","[argentine alternative rock, argentine alterna...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
df_related_genres = df_artists['related_genres'].apply(lambda x: pd.Series(x).value_counts())
df_related_genres.columns = ['related_genre: '+c for c in df_related_genres.columns]
df_related_genres.fillna(0, inplace=True)
df_related_genres.head()

,related_genre: rap,related_genre: trap,related_genre: pop rap,related_genre: hip hop,related_genre: southern hip hop,related_genre: underground hip hop,related_genre: atl hip hop,related_genre: melodic rap,related_genre: vapor trap,related_genre: conscious hip hop,...,related_genre: minimal tech house,related_genre: bboy,related_genre: filter house,related_genre: south african hip hop,related_genre: electro swing,related_genre: louisville indie,related_genre: experimental bass,related_genre: south african electronic,related_genre: nz electronic,related_genre: latincore
Travis Scott,20.0,17.0,16.0,15.0,10.0,6.0,4.0,4.0,3.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Martin Garrix,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Los Fabulosos Cadillacs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BROCKHAMPTON,13.0,0.0,3.0,13.0,0.0,8.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Duki,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
df_artists = df_artists.join(df_related_genres)
df_artists.head()

,artist_id,followers,genres,popularity,related_artists,related_genres,genre: alternative r&b,genre: alternative rock,genre: argentine alternative rock,genre: argentine hip hop,...,related_genre: minimal tech house,related_genre: bboy,related_genre: filter house,related_genre: south african hip hop,related_genre: electro swing,related_genre: louisville indie,related_genre: experimental bass,related_genre: south african electronic,related_genre: nz electronic,related_genre: latincore
Travis Scott,0Y5tJX1MQlPlqiwlOH1tJY,7663003,[rap],98,"[A$AP Rocky, Chief Keef, KIDS SEE GHOSTS, Push...","[east coast hip hop, hip hop, pop rap, rap, tr...",0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Martin Garrix,60d24wfXkVzDSfLS6hyCjZ,12531782,"[big room, edm, pop, progressive house, tropic...",88,"[AREA21, Brooks, Deorro, Lucas & Steve, KAAZE,...","[big room, edm, big room, deep big room, edm, ...",0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Los Fabulosos Cadillacs,2FS22haX3FYbyOsUAkuYqZ,1668356,"[argentine rock, latin alternative, latin rock...",73,"[Los Pericos, Vicentico, Aterciopelados, Los A...","[argentine reggae, argentine rock, latin alter...",0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BROCKHAMPTON,1Bl6wpkWCQ4KVgnASpvzzA,1174109,"[boy band, hip hop, rap]",85,"[Kevin Abstract, Matt Champion, JPEGMAFIA, Ame...","[hip hop, lgbtq+ hip hop, rap, bedroom pop, al...",0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Duki,1bAftSH8umNcGZ0uyV7LMg,2700103,"[argentine hip hop, trap argentino]",84,"[Uriel Natero, Bhavi, Khea, Lit Killah, Seven ...","[argentine hip hop, trap argentino, argentine ...",0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
